In [1]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime, timedelta
import pytz

In [2]:
# Initialize MT5 connection
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

In [3]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

In [4]:
# Define timezones
local_tz = pytz.timezone('Asia/Singapore')  # GMT+8
mt5_tz = pytz.timezone('Europe/Moscow')  # GMT+3 (MT5 server time)

# Get the current time in local timezone (GMT+8)
now_local = datetime.now(local_tz)

# Convert the local time to MT5 server time (GMT+3)
now_mt5 = now_local.astimezone(mt5_tz)

# Calculate the start and end times for fetching data
end_time = now_mt5 - timedelta(minutes=2)
start_time = end_time - timedelta(days=7)

# Get OHLC data for the last 7 days
rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)

# Create a DataFrame
ohlc_df = pd.DataFrame(rates)
ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
ohlc_df['time'] = ohlc_df['time'].dt.tz_localize('UTC').dt.tz_convert(mt5_tz)

In [5]:
ohlc_df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
ohlc_df = ohlc_df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)
ohlc_df.tail()

,time,open,high,low,close
7194,2024-08-08 08:29:00+03:00,1.09285,1.09295,1.09284,1.09295
7195,2024-08-08 08:30:00+03:00,1.09296,1.09303,1.09293,1.09295
7196,2024-08-08 08:31:00+03:00,1.09295,1.09311,1.09295,1.09306
7197,2024-08-08 08:32:00+03:00,1.09307,1.09324,1.09306,1.09323
7198,2024-08-08 08:33:00+03:00,1.09323,1.09326,1.09314,1.09315


In [6]:
#function to fetch the latest 1 min ohlc data
def fetch_latest_data():
    now_local = datetime.now(local_tz)
    now_mt5 = now_local.astimezone(mt5_tz)
    end_time = now_mt5 - timedelta( minutes=1)
    start_time = now_mt5 - timedelta( minutes=2)

    rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
    one_df = pd.DataFrame(rates)
    one_df = pd.DataFrame(rates)
    one_df['time'] = pd.to_datetime(one_df['time'], unit='s')
    one_df['time'] = one_df['time'].dt.tz_localize('UTC').dt.tz_convert(mt5_tz)
    one_df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    one_df = one_df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)

    
    # df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
    # target_timezone = pytz.timezone('Etc/GMT-3')
    # df['time'] = df['time'].dt.tz_convert(target_timezone)
    # df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
    # # df.set_index('time', inplace=True)
    # df.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    # df = df.reindex(labels=["time","open", "high", "low", "close"], axis = 1)
    return one_df

In [ ]:
# Infinite loop to fetch data every minute
try:
    while True:
        new_data = fetch_latest_data()
        ohlc_df = pd.concat([ohlc_df,new_data],ignore_index=True)
        ohlc_df.drop_duplicates(subset="time", keep="last", ignore_index= True)
        print(ohlc_df)
        
        # Wait for 60 seconds before fetching new data
        time.sleep(60)
except KeyboardInterrupt:
    print("Process interrupted by user")

# Shutdown MT5 connection
mt5.shutdown()


                          time     open     high      low    close
0    2024-08-01 08:34:00+03:00  1.08277  1.08280  1.08276  1.08280
1    2024-08-01 08:35:00+03:00  1.08280  1.08287  1.08278  1.08284
2    2024-08-01 08:36:00+03:00  1.08283  1.08288  1.08281  1.08284
3    2024-08-01 08:37:00+03:00  1.08284  1.08291  1.08280  1.08281
4    2024-08-01 08:38:00+03:00  1.08282  1.08290  1.08281  1.08290
...                        ...      ...      ...      ...      ...
7195 2024-08-08 08:30:00+03:00  1.09296  1.09303  1.09293  1.09295
7196 2024-08-08 08:31:00+03:00  1.09295  1.09311  1.09295  1.09306
7197 2024-08-08 08:32:00+03:00  1.09307  1.09324  1.09306  1.09323
7198 2024-08-08 08:33:00+03:00  1.09323  1.09326  1.09314  1.09315
7199 2024-08-08 08:34:00+03:00  1.09315  1.09322  1.09315  1.09321

[7200 rows x 5 columns]
                          time     open     high      low    close
0    2024-08-01 08:34:00+03:00  1.08277  1.08280  1.08276  1.08280
1    2024-08-01 08:35:00+03:00  1.082